## cnn 训练

## CNN 使用字符分割的验证码识别流程

### 1. 图像预处理‌

-‌‌ ‌灰度化与二值化‌：将彩色验证码转为灰度图，并通过阈值化（如 Otsu 算法）生成二值图像，突出字符轮廓‌35。
‌去噪与形态学优化‌：使用高斯模糊消除噪声，通过膨胀（Dilation）和腐蚀（Erosion）连接断裂字符‌35。
‌示例代码‌：
python
Copy Code


### 2. 字符分割‌

‌投影法‌：根据水平/垂直像素投影分布定位字符边界，适合字符间距较大的场景‌23。
‌连通域分析‌：提取图像中的连通区域，筛选面积和宽高符合字符特征的区域‌24。
‌区域生长法‌：从种子点扩展，合并相邻像素形成字符区域，适用于无粘连字符‌3。
‌粘连字符处理‌：对复杂粘连场景，需结合形态学操作或基于深度学习的语义分割（如 U-Net）‌34。
‌3. 数据准备‌
‌单字符标注‌：将分割后的单字符图像按类别（0-9、A-Z、a-z）存储至对应文件夹‌12。
‌数据增强‌：对单字符图像进行旋转（±10°）、平移（±5%）、缩放（0.9–1.1 倍）以扩充数据集‌25。
‌标准化输入‌：统一图像尺寸（如 32×32 像素），归一化像素值至 [0,1]‌13。
‌4. 单字符 CNN 模型训练‌
‌网络结构‌：
python
Copy Code
model = Sequential([  
    Conv2D(32, (3,3), activation='relu', input_shape=(32,32,1)),  
    MaxPooling2D((2,2)),  
    Conv2D(64, (3,3), activation='relu'),  
    Flatten(),  
    Dense(62, activation='softmax')  # 输出 62 类字符概率  
])  
‌损失函数与优化‌：使用交叉熵损失（CategoricalCrossentropy）和 Adam 优化器‌12。
‌训练策略‌：批量大小设为 32–64，训练轮数（Epochs）约 50–100，验证集准确率可达 90% 以上‌14。
‌5. 后处理与结果组合‌
‌序列还原‌：将单字符预测结果按原始验证码顺序拼接，形成完整字符串‌14。
‌纠错规则‌：结合字符上下文（如固定长度、字符类型）修正异常结果（如字母与数字混合场景）‌

In [ ]:
import cv2  
import numpy as np  
from keras.models import load_model  

def preprocess_captcha(image_path):  
    # 1. 灰度化与二值化  
    img = cv2.imread(image_path)  
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
    binary = cv2.adaptiveThreshold(  
        gray, 255,  
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,  
        cv2.THRESH_BINARY_INV, 11, 2  
    )  

    # 2. 去噪与形态学优化  
    blurred = cv2.GaussianBlur(binary, (3, 3), 0)  
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))  
    dilated = cv2.dilate(blurred, kernel, iterations=1)  
    eroded = cv2.erode(dilated, kernel, iterations=1)  

    # 3. 二次二值化与降噪  
    _, thresh = cv2.threshold(eroded, 127, 255, cv2.THRESH_BINARY)  
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
    for cnt in contours:  
        if cv2.contourArea(cnt) < 50:  
            cv2.drawContours(thresh, [cnt], -1, 0, -1)  

    return thresh  



def split_captcha(image_path):
    thresh = preprocess_captcha(image_path)  
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
    sorted_ctrs = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])  
    captchas = []  
    for i, ctr in enumerate(sorted_ctrs):  
        x, y, w, h = cv2.boundingRect(ctr)  
        roi = thresh[y:y+h, x:x+w]  
        if w > 10 and h > 10:  
            captchas.append(roi)  
    return captchas

def recognize_captcha(image_path):
    captchas = split_captcha(image_path)  
    result = []  
    for captcha in captchas:  
        _, captcha = cv2.threshold(captcha, 127, 255, cv2.THRESH_BINARY)  
        captcha = cv2.resize(captcha, (30, 30))  
        captcha = captcha.reshape(-1, 30, 30, 1)  
        prediction = model.predict(captcha)  
        result.append(chr(np.argmax(prediction) + 65))  
    return ''.join(result)


processed_image = preprocess_captcha(image_path)
cv2.imshow('Processed Image', processed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_cnn_model(input_shape=(28, 28, 1), num_classes=36):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # Dropout层，防止过拟合
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img

def load_and_preprocess_image(image_path):
    img = load_img(image_path, color_mode='grayscale', target_size=(28, 28))
    img = img_to_array(img) / 255.0  # 数据归一化
    return img

In [ ]:
model = build_cnn_model(input_shape=(28, 28, 1), num_classes=36)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10)  # train_images 和 train_labels 需要提前准备